In [ ]:
''' This notebook was run in google colabs because it ran faster, seemed to have fewer errors during BLAST api calls, 
and it was easy to directly save the results into a google drive folder for immediate sharing. It requires a google worksheet with ensembl IDs (Cell 2, line 4, worksheet=...).
If the ids are not in the 5th column, alter the column used (Cell 2, line 7, GeneIds =...)   '''

!pip install -q gget
import gget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 42.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.2/25.2 MB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import os

from google.colab import files, auth
from google.auth import default
import gspread


In [ ]:
# authorize accessing googlesheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

#open the google sheet with dREs
worksheet = gc.open('Known STR disease loci - from Shi et al. Nat comm. 2023 supplementary data 2').sheet1

# get_all_values gives a list of rows.
rows = worksheet.get_all_values()

# Convert to a DataFrame and get Ensembl IDs
dREs = pd.DataFrame.from_records(rows)
GeneIDs = dREs.loc[1:,5]

In [ ]:
#clean up Gene Ids
GeneIDs.replace('', np.nan, inplace=True)
GeneIDs.dropna(inplace = True)
GeneIDs.drop_duplicates(inplace = True)


In [ ]:
output_folder_name= "dRE_BlastResults"
output_folder_path='./drive/MyDrive/' + output_folder_name
try:
    os.mkdir(output_folder_path)
except OSError as e:
    print("Directory exists")


Directory exists


In [ ]:
#BLASTing
num_hits =[]
for Eid in GeneIDs:

  blast_results = None
  output_file_name = output_folder_path + '/' +Eid +'.csv'
  seq = gget.seq(Eid, translate=True)

  if not seq: #skip if there is no associated peptide
    continue
  if os.path.isfile(output_file_name): #skip if the BLAST search was already run
    continue

  while blast_results is None: #if the BLAST search fails retry it
   blast_results = gget.blast(seq[1], program="blastp", database="nr" , limit="10000")

  blast_results.to_csv(output_file_name, encoding = 'utf-8-sig') #save BLAST results as a csv

ERROR:root:No UniProt amino acid sequences were found for these ID(s).
ERROR:root:No UniProt amino acid sequences were found for these ID(s).
ERROR:root:Search 86VA52U2013 failed; please try again and/or report to blast-help@ncbi.nlm.nih.gov.
